# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(" "))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open("ner-train.tsv.bz2", 'rt', encoding="utf8") as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open("ner-dev.tsv.bz2", 'rt', encoding="utf8") as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
import numpy as np
import pandas as pd

In [6]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    TP = len(gold.intersection(pred))    # Number of true positives
    precision = TP/len(pred)
    recall = TP/len(gold)
    f1 = 2*precision*recall / (precision + recall)
    
    print("Precision = {:.4f} || Recall = {:.4f} || F1 Score = {:.4f}".format(precision, recall, f1))

To test your code, you can run the following cell:

In [7]:
evaluation_report(set(range(3)), set(range(5)))

Precision = 0.6000 || Recall = 1.0000 || F1 Score = 0.7500


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [8]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    for i,s_id,sent,beg,end,label in df.itertuples():
        yield (s_id, beg, end)

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [9]:
print("First triple: {} \nLast triple: {}".format(list(gold_spans(df_dev))[0], list(gold_spans(df_dev))[-1]))

First triple: ('0946-000', 2, 3) 
Last triple: ('1161-010', 1, 3)


In [10]:
spans_dev_gold = set(gold_spans(df_dev))
print("Number of unique triples = {}".format(len(spans_dev_gold)))

Number of unique triples = 5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [11]:
def get_spacy_ner(df):
    ner = []
    current_sid = ""
    with nlp.disable_pipes("tagger", "parser"):
        for _,s_id,sentence,_,_,_ in df.itertuples():
            if s_id != current_sid:
                sent_nlp = nlp(sentence)
                for ent in sent_nlp.ents:
                    ner.append((s_id, ent.start, ent.end))
                current_sid = s_id
    return ner

spacy_ner = get_spacy_ner(df_dev)

In [12]:
len(spacy_ner)

8038

In [13]:
evaluation_report(set(gold_spans(df_dev)), spacy_ner)

Precision = 0.5153 || Recall = 0.7000 || F1 Score = 0.5936


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Write code that prints the false positives and the false negatives from the automatic prediction. Have a look at the output. What are your observations? How could you improve the result? Discuss these questions in a short text.

In [14]:
# Function to join strings with comma
def merge_se(x):
    return ", ".join(x["start_end"].astype(str))

# Function to compare gold standard and predictions to create consolidated DataFrame of errors
def get_errors(df, gold, pred):
    # Find false positives and negatives
    fp = set(pred).difference(gold)
    fn = set(gold).difference(pred)
    # tp = set(gold).intersection(pred)
    
    # Create sentence id: sentence dictionary 
    sentence_dict = {}
    for i,s_id,sentence,beg,end,label in df.itertuples():
        if s_id not in sentence_dict:
            sentence_dict[s_id] = sentence.split()
    
    # Create DataFrames for FP and FN entities
    fp = [(s_id, " ".join(sentence_dict[s_id][start:end])) for s_id, start, end in fp]
    fn = [(s_id, " ".join(sentence_dict[s_id][start:end])) for s_id, start, end in fn]        
    fp_df = pd.DataFrame(fp, columns=["sentence_id", "start_end"]).groupby("sentence_id").apply(merge_se).to_frame("FP")
    fp_df.reset_index(level=0, inplace=True)
    fn_df = pd.DataFrame(fn, columns=["sentence_id", "start_end"]).groupby("sentence_id").apply(merge_se).to_frame("FN")
    fn_df.reset_index(level=0, inplace=True)
    
    # Join DataFrames
    errors_df = fp_df.set_index('sentence_id').join(fn_df.set_index('sentence_id'), how="outer")
    errors_merged_df = df_dev[["sentence_id","sentence"]].drop_duplicates().set_index('sentence_id').join(errors_df, how="right")
    errors_merged_df.reset_index(level=errors_merged_df.index.names, inplace=True)

    return errors_merged_df

In [15]:
error_analysis = get_errors(df_dev, set(gold_spans(df_dev)), spacy_ner)

In [16]:
error_analysis.loc[error_analysis["FP"].notna(),].loc[1:5,:]

,sentence_id,sentence,FP,FN
1,0946-002,West Indian all-rounder Phil Simmons took four...,"four, Friday, 38, two days, 39",NaN
2,0946-004,After bowling Somerset out for 83 on the openi...,"83, the opening morning, 83, first, 296, 94, t...",NaN
3,0946-005,"Trailing by 213 , Somerset got a solid start t...","second, 213, 174",NaN
4,0946-007,"Hussain , considered surplus to England 's one...","82, one-day, his first championship century of...",NaN
5,0946-008,By the close Yorkshire had turned that into a ...,"24, four, 48, 119, five",Such


In [17]:
error_analysis.loc[error_analysis["FN"].notna(),].loc[150:,:]

,sentence_id,sentence,FP,FN
150,0958-027,CLEVELAND AT TEXAS,NaN,CLEVELAND
151,0958-029,BOSTON AT OAKLAND,NaN,OAKLAND
152,0958-030,BALTIMORE AT SEATTLE,NaN,SEATTLE
153,0958-034,MONTREAL 71 61 .538 12,71 61,MONTREAL
154,0958-035,FLORIDA 64 70 .478 20,"20, 64 70",FLORIDA
...,...,...,...,...
2005,1161-001,DHAKA 1996-08-31,NaN,DHAKA
2006,1161-002,Shares on the Dhaka Stock Exchange ( DSE ) may...,"the Dhaka Stock Exchange, Friday weekend","Dhaka Stock Exchange, DSE"
2007,1161-005,"Broker Khurshid Alam said : "" The market senti...",Broker Khurshid Alam,Khurshid Alam
2008,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",NaN,IDLC


#### Error analysis and NER improvements

1. We observe many FP which are numbers (sometimes in words); words like "first", "second", etc; years; dates; day of week; amounts of money; quantities of measurements like metres, litres, etc. These are general entities which are identified by Spacy but are not present in our gold standard. By not predicting these false positives, we can improve our prediction quality. This can be easily achieved by ignoring these entities and not including them in our predictions. They have the following labels: "DATE", "TIME", "PERCENT", "MONEY", "ORDINAL", "CARDINAL", "QUANTITY" and "LAW". This results in a 17.78 percentage points increase in F1 score to 0.7714.
2. We see some cases where an entity occurs within quotes or has 's at the end or has "--" at the beginning which are included in the span by Spacy but not so in the gold standard. We can specifically check for these cases and adjust the spans accordingly. This results in a further 0.71 percentage points increase in F1 score to 0.7785.
3. We also see some false negatives which occur because Spacy includes "the" as part of the span of an entity. For example, Spacy predicts "the Dhaka Stock Exchange" as an entity but the gold standard contains only "Dhaka Stock Exchange". We check if entities start with "the" and adjust the span accordingly. This results in a further 2.45 percentage points increase in F1 score to 0.8030.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to pre-process the data and/or post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [126]:
import re

def get_spacy_ner_2(df):
    ner = []
    current_sid = ""
    with nlp.disable_pipes("tagger", "parser"):
        for i,s_id,sentence,beg,end,label in df.itertuples():
            
            if s_id != current_sid:
                sent_nlp = nlp(sentence)
                for ent in sent_nlp.ents:
                    if not ent.label_ in ["DATE", "TIME", "PERCENT", "MONEY", "ORDINAL", "CARDINAL", "QUANTITY", "LAW"]:
                        ner_obj = (s_id, ent.start, ent.end)
                        
                        if ent.text.startswith("\" ") and ent.text.endswith(" \""):
                            ner_obj = (ner_obj[0], ner_obj[1]+1, ner_obj[2]-1)
                        elif ent.text.startswith("\" "):
                            ner_obj = (ner_obj[0], ner_obj[1]+1, ner_obj[2])
                        elif ent.text.endswith(" \""):
                            ner_obj = (ner_obj[0], ner_obj[1], ner_obj[2]-1)
                        
                        if ent.text.endswith(" \'s"):
                            ner_obj = (ner_obj[0], ner_obj[1], ner_obj[2]-1)
                            
                        if ent.text.startswith("-- "):
                            ner_obj = (ner_obj[0], ner_obj[1]+1, ner_obj[2])
                            
                        if ent.text.startswith("the "):
                            ner_obj = (ner_obj[0], ner_obj[1]+1, ner_obj[2])
                        
                        ner.append(ner_obj)
                current_sid = s_id
    return ner

spacy_ner_improved = get_spacy_ner_2(df_dev)
evaluation_report(set(gold_spans(df_dev)), spacy_ner_improved)

Precision = 0.8971 || Recall = 0.7267 || F1 Score = 0.8030


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the predicted start and end positions for each token span. As the `label` of each span, you can use the special value `--NME--`.

In [105]:
set(spacy_imp_spans["label"])

{'--NME--'}

In [90]:
"US" in unique_kb_mentions

True

In [19]:
# DataFrame of sentences and IDs
unique_sent_df = df_dev[["sentence_id","sentence"]].drop_duplicates().set_index('sentence_id')

# DataFrame of post-processed Spacy NER predictions
spacy_imp_spans = pd.DataFrame(spacy_ner_improved, columns=["sentence_id", "beg", "end"]).set_index('sentence_id')

# Join DataFrames to get DataFrame similar to gold standard
spacy_imp_spans = unique_sent_df.join(spacy_imp_spans, how="inner")
spacy_imp_spans.reset_index(level=spacy_imp_spans.index.names, inplace=True)
spacy_imp_spans["label"] = "--NME--"
spacy_imp_spans

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,3,--NME--
1,0946-001,LONDON 1996-08-30,0,1,--NME--
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,--NME--
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,--NME--
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,--NME--
...,...,...,...,...,...
4788,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,--NME--
4789,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,--NME--
4790,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,--NME--
4791,1161-009,The DSE all share price index closed 2.73 poin...,1,2,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [20]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for _,s_id,_,beg,end,label in df.itertuples():
        yield (s_id, beg, end, label)

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [21]:
spacy_improved_labels = [(s_id, beg, end, "_".join(sent.split()[beg:end])) for 
                         i,s_id,sent,beg,end,label in spacy_imp_spans.itertuples()]

In [22]:
evaluation_report(set(gold_mentions(df_dev)), spacy_improved_labels)

Precision = 0.3326 || Recall = 0.2694 || F1 Score = 0.2977


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on when you click on a link) as mentions of the entities (pages) that they link to. This allows us to harvest long lists over mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [23]:
with bz2.open("kb.tsv.bz2", 'rt', encoding="utf8") as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [24]:
df_kb.loc[df_kb["mention"] == "Sweden"]

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing stands first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [25]:
# TODO: Write code here to implement the "most probable entity" method.
def mp_entity(labelled_tuples):
    mp_entity_labels = []
    for s_id, beg, end, label in labelled_tuples:
        # Find entities corresponding to mention
        candidate_mentions = df_kb.loc[df_kb["mention"] == label.replace("_", " ")]
        
        # Find most probable entity among the candidates
        if candidate_mentions.shape[0] == 0:
            mpe = "--NME--"
        else:
            mpe = candidate_mentions.reset_index().loc[0, "entity"]
        mp_entity_labels.append((s_id, beg, end, mpe))
    
    return mp_entity_labels

most_probable_labels = mp_entity(spacy_improved_labels)
evaluation_report(set(gold_mentions(df_dev)), most_probable_labels)

Precision = 0.6914 || Recall = 0.5601 || F1 Score = 0.6189


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [26]:
with bz2.open("contexts.tsv.bz2") as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as a bag of words containing the mention itself, as well as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [27]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in 1970 and the second in ...
1,1970,FIFA_World_Cup,America 1975 and during the 1970 and 1978 Worl...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the 1990 World Cup
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the 1990 W...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the 1990 World Cup The...


In [28]:
df_kb.loc[df_kb["mention"] == "Lincoln"]

,mention,entity,prob
10559,Lincoln,"Lincoln,_Nebraska",0.748718
10560,Lincoln,Abraham_Lincoln,0.220513
10561,Lincoln,Lincoln_City_F.C.,0.030769


From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [29]:
df_contexts.context[df_contexts.mention == "Lincoln"]

41465    Nebraska Concealed Handgun Permit In Lincoln m...
41466    Lazlo restaurants are located in Lincoln and O...
41467    California Washington Overland Park Kansas Lin...
41468    City Missouri Omaha Nebraska and Lincoln Nebra...
41469    by Sandhills Publishing Company in Lincoln Neb...
                               ...                        
41609                                Lincoln Leyton Orient
41610               English division three Swansea Lincoln
41611    league membership narrowly edging out Lincoln ...
41612                                    Lincoln Cambridge
41613                                              Lincoln
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** To tune the performance of your method, you can try to tweak the behaviour of the vectorizer (for example, should it apply lowercasing or not?) and the width of the window from which you are extracting context tokens at prediction time.

**Hint 3:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [80]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from collections import Counter

# MultinomialNaiveBayes model pipeline
# binary=True, alpha=0.5 options led to slightly better results
cnt_vectorizer = CountVectorizer(lowercase=False, binary=True)
mnb = MultinomialNB(alpha=0.5)
mnb_pipeline = Pipeline([("cnt_vectorizer", cnt_vectorizer), ("mnb", mnb)])

mnb_model_dict = {}

# Count of entities associated with each mention
kb_mentions_counter = Counter(df_kb["mention"])

# Mentions to store in dictionary
unique_spacy_mentions = set([label.replace("_", " ") for _, _, _, label in spacy_improved_labels])

for mention in unique_spacy_mentions:
    if kb_mentions_counter[mention]>1:       # Mention contains multiple associated entities
        # Extract entities and contexts corresponding to the mention
        candidate_entities = df_kb.loc[df_kb["mention"] == mention]
        contexts_mentions = df_contexts.loc[df_contexts.mention == mention]
        
        # Class priors sorted in alphabetical order of entities
        class_order = np.argsort(candidate_entities["entity"]).values
        sorted_class_priors = candidate_entities["prob"].values[class_order]
        
        # Create model using contexts corresponding to the mention
        mnb_model_dict[mention] = clone(mnb_pipeline).set_params(mnb__class_prior=sorted_class_priors)
        mnb_model_dict[mention].fit(contexts_mentions["context"], contexts_mentions["entity"])
        
    elif kb_mentions_counter[mention]==1:    # Mention contains only one associated entity
        candidate_entities = df_kb.loc[df_kb["mention"] == mention]
        mnb_model_dict[mention] = candidate_entities.reset_index().loc[0, "entity"]
        
    elif kb_mentions_counter[mention]==0:    # Mention contains no associated entity
        mnb_model_dict[mention] = "--NME--"

In [81]:
# Function to disambiguate mentions and link to entities using a MultinomialNaiveBayes model
def context_sensitive_disambiguate(df, labelled_tuples, df_kb, df_contexts):
    disamb_entity_labels = []
    
    # Create a sentence dictionary
    sentence_dict = {}
    for i,s_id,sentence,beg,end,label in df.itertuples():
        if s_id not in sentence_dict:
            sentence_dict[s_id] = sentence.split()
    
    for s_id, beg, end, label in labelled_tuples:
        mention = label.replace("_", " ")
        if isinstance(mnb_model_dict[mention], str):
            disamb_entity = mnb_model_dict[mention]
        else:
            context = " ".join(sentence_dict[s_id][max(0, beg-5):min(len(sentence_dict[s_id]), end+5)])
            disamb_entity = mnb_model_dict[mention].predict([context])[0]
        
        disamb_entity_labels.append((s_id, beg, end, disamb_entity))
    
    return disamb_entity_labels

csd_labels = context_sensitive_disambiguate(df_dev, spacy_improved_labels, df_kb, df_contexts)

In [82]:
evaluation_report(set(gold_mentions(df_dev)), csd_labels)

Precision = 0.7087 || Recall = 0.5741 || F1 Score = 0.6344


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1. Published systems report a larger impact of context-sensitive disambiguation. Feel free to think about what could explain the relatively minor impact that we see here!

We see only 1.55 percentage point improvement in F1 score. We think that this maybe because of the amount of data that we use to disambiguate. We have very less data for each mention to train a multinomial naive bayes classifier. Maybe a discriminative model might work better with such less amount of data. Also, the words which help in disambiguation might occur outside the 5 word boundary that we are considering here. 

**This was the last lab in the Text Mining course. Well done!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>